In [1]:
import sys
sys.path.append('../src')

In [2]:
from metrics import calculate_false_alarm, calculate_delay, calculate_precision, compute_metrics
from loading import *
from visualization import *
from features import *
from helpers import *
from filters import *
import pandas as pd
import pickle

from matplotlib import pyplot as plt
%matplotlib inline

# Score evaluation automation function

In [35]:
def compute_patient_score(data_path, feature_functions, parameters, save=True):
    print("Loading data")
    print()
    patient_data = load_patient_seizures(data_path, parameters['patient_id'],all_seizures=True, delta=parameters['delta'])
    print()

    eegs = patient_data['eegs']
    ranges = patient_data['ranges']
    seizure_start_s = patient_data["infos"]["seizure_start"]
    seizure_end_s = patient_data["infos"]["seizure_end"]
    
    print("Finding max power EEG signal ...")
    max_power_eeg = top_signals(eegs, ranges, top=1, id_seizure=None)
    
    window_size = parameters['win_size']
    step_size = parameters['step_size']
    
    print("Computing new seizure indexes ...")
    indexes = convert_sample_ranges_to_window_ranges(ranges,window_size,step_size,parameters['num_samples'])
    

    if save:
        result_df = pd.DataFrame(columns=['patient_id', 'seizure_id', 'feature', 'best_score', 'threshold', 'overall_score']+list(parameters.keys()))
        saving_path = data_path + 'results/' + str(patient_id) + '/'
        os.makedirs(saving_path, exist_ok=True)

    for feature in feature_functions.keys():

        print("Computing feature {} ...".format(feature))
        feature_signals = calculate_feature(eegs, window_size, step_size, feature_functions[feature], sensor=max_power_eeg[0])
        
        print("Merging data ...")
        all_data_window_ranges, all_feature_data = merge_all_data(feature_signals, indexes)
        
        print("Computing metrics ...")
        thresholds, false_alarms, delays, precisions = compute_metrics(all_feature_data, all_data_window_ranges, window_size, step_size)
        
        if save:
            print("Saving results data ...")
            with open(saving_path + 'patient-{0}-{1}-data.pkl'.format(patient_id, feature), 'wb') as f0:
                pickle.dump(feature_signals, f0)
            with open(saving_path + 'patient-{0}-{1}-false-alarms.pkl'.format(patient_id, feature), 'wb') as f1:
                pickle.dump(false_alarms, f1)
            with open(saving_path + 'patient-{0}-{1}-delays.pkl'.format(patient_id, feature), 'wb') as f2:
                pickle.dump(delays, f2)
            with open(saving_path + 'patient-{0}-{1}-precisions.pkl'.format(patient_id, feature), 'wb') as f3:
                pickle.dump(precisions, f3)
        
        print("Computing scores")
        print()
        feature_scores = compute_score_on_seizures(thresholds, false_alarms, precisions, delays, parameters['weights'])
        overall_score = compute_overall_score(feature_scores)
        
        for i, feature_score in enumerate(feature_scores):
            new_row = {'seizure_id':i+1, 'feature':feature, 'best_score': feature_score[0], 'threshold': feature_score[1], 'overall_score': overall_score}
            new_row = {**parameters, **new_row}
            result_df = result_df.append(new_row, ignore_index=True)
        
        print("Feature scores", feature_scores)
        print("Overall scores", overall_score)
        print()
    
    if save:
        print("Saving result dataframe")
        result_df.to_csv(saving_path + 'results.csv', index=False)
    

In [36]:
data_path = "../data/"
feature_functions = {'energy':calculate_energy, 'line_length':calculate_line_length, 'moving_avg':calculate_moving_avg}
parameters = {"patient_id":2 , "delta":1, "win_size":128, "step_size":64, "num_samples":1843200, "weights":(1,1,1)}

In [37]:
compute_patient_score(data_path, feature_functions, parameters)

Loading data

Seizure starts (s):  [[824604.12214844]
 [837451.78589844]]
Seizure ends (s):  [[824690.37769531]
 [837541.65542969]]
Duration (s):  [[86.25554687]
 [89.86953125]]
Sampled frequency (Hz):  512
Start/end [824604.12214844] [824690.37769531]
Start/end [837451.78589844] [837541.65542969]
EEG files:  [['../data/ID02/ID02_229h.mat', '../data/ID02/ID02_230h.mat', '../data/ID02/ID02_231h.mat'], ['../data/ID02/ID02_232h.mat', '../data/ID02/ID02_233h.mat', '../data/ID02/ID02_234h.mat']]
Hour of seizure:  [array([229, 230, 231]), array([232, 233, 234])]
Samples ranges:  [(104510, 148673), (1152914, 1198927)]

Finding max power EEG signal ...
Computing new seizure indexes ...
Computing feature energy ...
Merging data ...
Computing metrics ...
Saving results data ...
Computing scores

Max score for seizure 0 is 0.9895441972780928 for t=0.058
FA: 9636
Delay: 9.125316851279559
Precision: 1.0

Max score for seizure 1 is 0.9998573218155242 for t=0.184
FA: 2683
Delay: 0.3750130212854613
Pr